# Removing variables

- Importing required libraries

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [69]:
train_df = pd.read_csv('train.csv')
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [76]:
train_df['SalePrice'] = train_df['SalePrice'].apply(lambda x: math.log(x))

- We filtered the data according to 4 basic criterion: Relevance, Missing data, balance of the data and repeated columns


In [6]:
insufficient_data = ['Alley', 'Fence', 'MiscFeature', 'PoolQC', 'FireplaceQu', 
                     'LotFrontage']

imbalanced_data = ['3SsnPorch','BsmtCond', 'GarageQual', 'GarageCond', 'Heating', 'LandContour', 
                   'LandSlope', 'LowQualFinSF','PoolArea', 'RoofMatl', 'Street', 'Utilities']

nonsense_data = ['Condition1', 'Condition2', 'Functional', 'GarageType', 'GarageYrBlt', 
                 'BsmtHalfBath','LotConfig', 'LotShape', 'PavedDrive', 'MoSold', 'SaleCondition',
                 'YrSold','Id']

extra_data = ['GarageCars', 'GarageFinish', 'BsmtFullBath', 'YearBuilt']

train_df.drop(insufficient_data + imbalanced_data + nonsense_data + extra_data, axis = 1,
             inplace = True)

In [86]:
train_df['LandContour'].value_counts()

Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

- second_layer_checks to see if we missed anything with our logic

In [8]:
def correlation(dataset, threshold):
    corr_cols = []
    corr_matrix = dataset.corr(method = 'pearson')
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold):
                corr_cols.append((corr_matrix.columns[i],corr_matrix.columns[j], corr_matrix.iloc[i,j]))
    return corr_cols
#     print(dataset)

correlation(train_df,0.75)

[('1stFlrSF', 'TotalBsmtSF', 0.8195299750050355),
 ('TotRmsAbvGrd', 'GrLivArea', 0.8254893743088377),
 ('SalePrice', 'OverallQual', 0.7909816005838047)]

- Might not need to drop because correlations between variables do not really affect the predictive capabilities of the model
- Consider looking at variables with very low correlation to the SalesPrice: Potentially can drop

In [9]:
second_check = ['TotRmsAbvGrd', 'TotalBsmtSF']
train_df.drop(second_check, axis =1, inplace = True)
# train_df.shape

# Dealing with missing data

In [10]:
missing_data = train_df.isna().sum() / train_df.shape[0]
missing_data[missing_data > 0]

MasVnrType      0.005479
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
dtype: float64

In [11]:
train_df[(train_df['BsmtFinType1'].isna() == False) & (train_df['BsmtFinType2'].isna() == True)][['BsmtFinType1', 'BsmtFinType2']]

,BsmtFinType1,BsmtFinType2
332,GLQ,NaN


In [12]:
print(train_df['Electrical'].value_counts())
print()
print(train_df['MasVnrType'].value_counts())

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64


- We should not remove the missing data with no basement. Likely to not be missing at random. We introduce new category called 'No Basement'
- For electrical, because we only have 1 missing row, we think it is reasonable to use the mode of the variable to fill this missing value
- For MasVnrType and MasVnrArea, we filled NA to assume that there is no MasVnr. Easiest way of dealing with inputting categorical for now

In [13]:
train_df.update(train_df[['BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna('No Basement'))
train_df['Electrical'] = train_df['Electrical'].fillna('SBrkr')
train_df['MasVnrType'] = train_df['MasVnrType'].fillna('None')
train_df['MasVnrArea'] = train_df['MasVnrArea'].fillna(0)

- Methods to input categorical: 1. Use the mode (introduce bias), 2. Missing values as new category, 3. Use classification algorithm to predict like Logistic regression / KNN

# Further Cleaning

In [14]:
train_df.corr(method='pearson').style.background_gradient(cmap='coolwarm')

,MSSubClass,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,MiscVal,SalePrice
MSSubClass,1,-0.139781,0.0326277,-0.0593158,0.040581,0.023573,-0.0698357,-0.0656486,-0.140759,-0.251758,0.307886,0.0748532,0.131608,0.177354,-0.023438,0.281721,-0.0455693,-0.0986715,-0.0125794,-0.00610012,-0.0120366,-0.0260302,-0.00768329,-0.0842841
LotArea,-0.139781,1,0.105806,-0.00563627,0.0137884,0.103321,0.214103,0.11117,-0.00261836,0.299475,0.0509859,0.263116,0.126031,0.0142595,0.11969,-0.0177839,0.271364,0.180403,0.171698,0.0847738,-0.0183397,0.0431604,0.0380677,0.263843
OverallQual,0.0326277,0.105806,1,-0.0919323,0.550684,0.407252,0.239666,-0.0591187,0.308159,0.476224,0.295493,0.593007,0.5506,0.273458,0.101676,-0.183882,0.396765,0.562022,0.238923,0.308819,-0.113937,0.0648864,-0.0314062,0.790982
OverallCond,-0.0593158,-0.00563627,-0.0919323,1,0.0737415,-0.125694,-0.0462309,0.0402292,-0.136841,-0.144203,0.0289421,-0.0796859,-0.194149,-0.0607693,0.0129801,-0.0870009,-0.02382,-0.151521,-0.0033337,-0.0325888,0.0703562,0.0548105,0.0687768,-0.0778559
YearRemodAdd,0.040581,0.0137884,0.550684,0.0737415,1,0.176529,0.128451,-0.0677585,0.181133,0.240379,0.140024,0.287389,0.439046,0.183331,-0.0405809,-0.149598,0.112581,0.3716,0.205726,0.226298,-0.193919,-0.03874,-0.0102862,0.507101
MasVnrArea,0.023573,0.103321,0.407252,-0.125694,0.176529,1,0.261256,-0.0713296,0.113862,0.33985,0.1738,0.388052,0.272999,0.199108,0.102775,-0.0384501,0.247015,0.370884,0.159991,0.122528,-0.109907,0.0622477,-0.0295122,0.472614
BsmtFinSF1,-0.0698357,0.214103,0.239666,-0.0462309,0.128451,0.261256,1,-0.0501174,-0.495251,0.445863,-0.137079,0.208171,0.0585431,0.00426242,-0.107355,-0.0810069,0.260011,0.29697,0.204306,0.111761,-0.102303,0.0620206,0.00357147,0.38642
BsmtFinSF2,-0.0656486,0.11117,-0.0591187,0.0402292,-0.0677585,-0.0713296,-0.0501174,1,-0.209294,0.0971174,-0.0992603,-0.00963989,-0.0764439,-0.0321478,-0.0157281,-0.0407512,0.0469207,-0.0182266,0.0678983,0.00309256,0.0365433,0.0888713,0.00493978,-0.0113781
BsmtUnfSF,-0.140759,-0.00261836,0.308159,-0.136841,0.181133,0.113862,-0.495251,-0.209294,1,0.317987,0.00446909,0.240257,0.288886,-0.0411175,0.166643,0.0300859,0.0515749,0.183303,-0.00531642,0.129005,-0.00253785,-0.0125793,-0.0238366,0.214479
1stFlrSF,-0.251758,0.299475,0.476224,-0.144203,0.240379,0.33985,0.445863,0.0971174,0.317987,1,-0.202646,0.566024,0.380637,-0.119916,0.127401,0.0681006,0.410531,0.489782,0.235459,0.211671,-0.0652917,0.0887581,-0.0210957,0.605852


- MSSubClass is a qualitative variable cannot be seen as quantitative
- Dropping SalePrice from our cleaning because it is the dependent variable

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [16]:
train_df['MSSubClass'].value_counts()
train_df = train_df.astype({'MSSubClass': 'object'})
y = train_df['SalePrice']
train_df.drop('SalePrice', axis = 1, inplace= True)

- Qualitative variables have to encoded using one hot encoding, so that you can run regression
- Quantitative variables have to be standardised so that you can use Regularisation effectively

In [17]:
qual_variables = list(train_df.columns[train_df.dtypes == 'object'])
qual_variables =pd.get_dummies(train_df[qual_variables], prefix = qual_variables, columns =  qual_variables)
qual_variables.head()

,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,...,KitchenQual_TA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
quant_variables = list(train_df.columns[train_df.dtypes != 'object'])
standardized_values = preprocessing.StandardScaler().fit_transform(train_df[quant_variables])
standardized_values = pd.DataFrame(standardized_values, columns = quant_variables)

- X_model_1 contains variables used for the original linear regresson model (without standardized values)
- X_model_2 contains standardised values

In [19]:
X_model_1= pd.merge(train_df[quant_variables], qual_variables, left_index = True, right_index = True)
X_model_2 = pd.merge(standardized_values, qual_variables, left_index = True, right_index = True)

- Comparing the results of standardisation. Seeing if it is similar to manual application of formula

In [20]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_model_2, y, test_size = 0.20, random_state = 10)

In [21]:
print((8450 - (sum(X_model_1['LotArea'])/len(X_model_1))) / (X_model_1['LotArea'].std()))
print(X_model_2['LotArea'][0])

-0.20707075668205202
-0.20714170777431132


# Running Linear Regression model

- Small difference due to standardisation

In [22]:
# lr = LinearRegression()
# lr.fit(X_train_1,y)
# train_score = lr.score(X_train_1,y)
# print(train_score)

lr2 = LinearRegression().fit(X_train_2,y_train_2)
train_score = lr2.score(X_train_2,y_train_2)
y_pred = lr2.predict(X_test_2)
y_pred_train = lr2.predict(X_train_2)
MSE1 = metrics.mean_squared_error(y_test_2, y_pred)
MAE1 = metrics.mean_absolute_error(y_test_2, y_pred)
MSE_train_1 = metrics.mean_squared_error(y_train_2, y_pred_train)
print('The TRAIN MSE for our Multivariate Linear Regression Model is', MSE_train_1)
print('The TEST MSE for our Multivariate Linear Regression Model is ', MSE1)
print('The TEST MAE for our Multivariate Linear Regression Model is ', MAE1)


The TRAIN MSE for our Multivariate Linear Regression Model is 774059650.8159246
The TEST MSE for our Multivariate Linear Regression Model is  2.6655253228187664e+29
The TEST MAE for our Multivariate Linear Regression Model is  44117608979745.82


In [63]:
rr = Ridge(alpha = 0.90)
rr.fit(X_train_2,y_train_2)
y_pred_2 = rr.predict(X_test_2)
y_pred_train_2 = rr.predict(X_train_2)
MSE2 = metrics.mean_squared_error(y_test_2, y_pred_2)
MAE2 = metrics.mean_absolute_error(y_test_2, y_pred_2)
MSE_train_2 = metrics.mean_squared_error(y_train_2, y_pred_train_2)
print('The TRAIN MSE for our Ridge Regression Model is ', MSE_train_2)
print('The TEST MSE for our Ridge Regression Model is ', math.sqrt(MSE2))
print('The TEST MAE for our Ridge Regression Model is ', MAE2)

The TRAIN MSE for our Ridge Regression Model is  777541651.0681487
The TEST MSE for our Ridge Regression Model is  26670.193543234298
The TEST MAE for our Ridge Regression Model is  18373.934467288207


# Running Random Forest Model

In [81]:
from sklearn.ensemble import RandomForestRegressor

In [61]:
regressor = RandomForestRegressor(n_estimators = 150, random_state = 10)
regressor.fit(X_train_2, y_train_2)
y_pred_RF = regressor.predict(X_test_2)
y_pred_train_RF = regressor.predict(X_train_2)

In [62]:
MSE_RF = metrics.mean_squared_error(y_test_2, y_pred_RF)
MSE_train_RF = metrics.mean_squared_error(y_train_2, y_pred_train_RF)
print('The TRAIN RMSE for our Model is ', round(math.sqrt(MSE_train_RF),2))
print('The TEST RMSE for our RF Model is ', round(math.sqrt(MSE_RF),2))

The TRAIN RMSE for our Model is  11006.72
The TEST RMSE for our RF Model is  29095.94


- Can consider iterating through various values of alpha, to get the lowest
- Can use Lasso regression to look at the weights given to the feature. Low weights indicative that the feature is not important